In [ ]:
from transformers import GenerationConfig, LlamaForCausalLM, LlamaTokenizer
import torch
from tqdm.notebook import tqdm, trange
import gc

In [ ]:
# MODEL_NAME = "meta-llama/Llama-2-7b-hf"
MODEL_NAME = "meta-llama/Llama-2-7b-chat-hf"

tokenizer = LlamaTokenizer.from_pretrained(MODEL_NAME)
model = LlamaForCausalLM.from_pretrained(
    MODEL_NAME,
    return_dict=True,
    # load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map="auto",
)
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
print(generation_config)
model = model.eval()

In [ ]:
def generate_response(prompt: str, max_new_tokens: int = 128, temperature: float = 0.001) -> str:
    encoding = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.inference_mode():
        outputs = model.generate(
            **encoding,
            max_new_tokens=max_new_tokens,
            temperature=0.001,
            generation_config=generation_config,

            do_sample = True,
        )
    answer_tokens = outputs[:, encoding.input_ids.shape[1] :]
    return tokenizer.decode(answer_tokens[0], skip_special_tokens=True)


def format_prompt(prompt, history='', Background=''):
    return f"""<s>[INST] <<SYS>>
{{ 
    {Background}
    {history}
}}
<</SYS>>
[/INST]  </s>
<s>[INST] {{ {prompt} }} [/INST]"""


In [ ]:
The_Topic = '"What if India becomes a superpower within next 10 years."'
User_A = "Tom"
User_B = "Jerry"
 
# Background_for_A = f'You are {User_A}. The topic of the discussion id {The_Topic}. Ask a question or write a response that answers the question of {User_B} according to the conversation history, or with a new information. Answer in one short sentence only and strictly, do not repeat what is already said in the conversation. If you have nothing new to add, reply with a thanks.'
# Background_for_B = f'You are {User_B}. The topic of the discussion id {The_Topic}. Ask a question or write a response that answers the question of {User_A} according to the conversation history, or with a new information. Answer in one short sentence only and strictly, do not repeat what is already said in the conversation. If you have nothing new to add, reply with a thanks.'

Background_for_A = f'You are {User_A}. The topic of the discussion is Communism. You are in favour of Communism. Ask a question or write a response that answers the question of {User_B} according to the conversation history, or with a new information. Answer in one short sentence only and strictly, do not repeat what is already said in the conversation. If you have nothing new to add, reply with a thanks.'
Background_for_B = f'You are {User_B}. The topic of the discussion is Communism. You are completely against the communism. Ask a question or write a response that answers the question of {User_A} according to the conversation history, or with a new information. Answer in one short sentence only and strictly, do not repeat what is already said in the conversation. If you have nothing new to add, reply with a thanks.'
Background = ''
the_output = ''

history = f'''
{User_A}: Hello, I am {User_A}
{User_B}:  Hello {User_A}, I am {User_B}
'''
prompt_to_A = f"Hello {User_A}, I am {User_B}"

for j in trange(1,30):
    try:
        prompt_to_B = generate_response(format_prompt(prompt_to_A, history, Background_for_A), 150, 0.001)
        if prompt_to_B[:len(User_A)] == User_A:
            history += prompt_to_B + '\n'
        else:
            history += f"{User_A}: " + prompt_to_B + '\n'

        prompt_to_A = generate_response(format_prompt(prompt_to_B, history, Background_for_B), 150, 0.001)
        if prompt_to_A[:len(User_B)] == User_B:
            history += prompt_to_A + '\n'
        else:
            history += f"{User_B}: " + prompt_to_A + '\n'
        
        if not j % 5:
            summary = generate_response(format_prompt("What are the keypoints of above chat and summary, in very very short.", history), 500)
            summary = '\n'.join(summary.strip().split('\n')[1:])
            print(summary)
            print("*"*50)
            history = "We know that\n" + summary +'\n' + f'''
{User_A}: Hello, I am {User_A}
{User_B}:  Hello {User_A}, I am {User_B}
'''
    except:
        break




In [ ]:
print(history)

In [ ]:
# summary = generate_response(format_prompt("What are the keypoints of above chat and summary, in very very short.", history), 500)
# newsummary = '\n'.join(summary.strip().split('\n')[1:])

# print(summary)

In [ ]:
# # newsummary = '\n'.join(summary.strip().split('\n')[1:])
# # print(newsummary)
# print(len(history))
# print(len(summary))

In [ ]:
####### SAVE TO TEXT FILE ########

# # Define the string you want to append to the file
# text_to_append = "This is additional text to append to the file."

# # Specify the file path where you want to append the text
# file_path = f"./{The_Topic}.txt"
# file_path = f"./AGI_tobeinvented.txt"

# # Open the file in append mode ('a' for appending)
# with open(file_path, 'a') as file:
#     # Write the string to the file
#     file.write("Conversation\n\n")
#     file.write(history)
#     file.write("Summary\n\n")
#     file.write(summary)

# # Close the file when you're done (the 'with' statement automatically does this)


In [ ]:
# history_for_A = '''
# User A: Hello, I am User A. 
# User B:  Hello User A, I am User B.
# '''

# history_for_B = '''
# User A: Hello, I am User A.
# User B: Hello User A, I am User B.
# '''

In [ ]:
# Fixed_Prefix = "Write a response that answers the request according to the conversation history below. Answer in one short sentence only."
# Background_for_A = 'Talk about Fake news with User B'
# Background_for_B = 'Talk about Fake news with User A'
# Background = ''
# the_output = ''
# Fixed_Prefix_len = len(Fixed_Prefix)
# import sys
# history = '''
# User A: Hello, I am User A.
# User B:  Hello!
# '''
# for j in range(1,50):
#     if not j % 5:    
#         summary = generate_response(format_prompt("What are the keypoints of above chat and summary, in very very short.", history), 500)
#         Background += '\n' + ''.join(Background.strip().split('\n')[1:])
#         history = '\n'
#     print(Background)
#     # print(history)
#     print(the_output)
#     print(j, "*" * 50)
#     while True:
#         prompt = input()
#         if len(prompt) == 0:
#             continue
#         else:
#             break
#     if prompt == 'quit':
#         break
#     the_output = generate_response(format_prompt(prompt, history), 200, 0.001)
#     history += "Request: " + prompt + '\n'
#     history += "Response: " + the_output + '\n\n'